# FairWorkflows execution demo

## Define the steps of your workflow
Each step should be its own function. Mark the function as such with the @fairstep decorator.

In [1]:
from fairworkflows import is_fairworkflow, is_fairstep

In [2]:
@is_fairstep(label='Addition')
def add(a:float, b:float) -> float:
    """Adding up numbers!"""
    return a + b

In [3]:
@is_fairstep(label='Subtraction')
def sub(a: float, b: float) -> float:
    """Subtracting numbers."""
    return a - b

In [4]:
@is_fairstep(label='Multiplication')
def mul(a: float, b: float) -> float:
    """Multiplying numbers."""
    return a * b

In [5]:
@is_fairstep(label='A strange step with little use')
def weird(a: float, b:float) -> float:
    """A weird function"""
    return a * 2 + b * 4
    

## Define your workflow using @fairworkflow
Now write a function which describes your workflow. Mark this function with the @fairworkflow decorator.

In [6]:
@is_fairworkflow(label='My Workflow')
def my_workflow(in1, in2, in3):
    """
    A simple addition, subtraction, multiplication workflow
    """
    t1 = add(in1, in2)
    t2 = sub(in1, in2)
    t3 = mul(weird(t1, in3), t2)
    return t3

## Create an instance of your workflow and display it

In [7]:
fw = my_workflow(1, 4, 3)
type(fw)

fairworkflows.fairworkflow.FairWorkflow

In [8]:
fw.display()

| workflow |
| --- |
| ![workflow workflow](control-workflow.svg) |

## Publish the (prospective) workflow
You may publish the workflow, and its steps, as nanopublications in the usual manner:

In [9]:
fw.publish_as_nanopub()

Published to http://purl.org/np/RANKki6Fh-QYTHJ39yhJNOEcQLZOJld1VZFEvaLeILgCA
Published concept to http://purl.org/np/RANKki6Fh-QYTHJ39yhJNOEcQLZOJld1VZFEvaLeILgCA#step
Published to http://purl.org/np/RAsljALsS_3aL57HIU-8tdyxYl3ESlSkQu3kCfWBxvZV0
Published concept to http://purl.org/np/RAsljALsS_3aL57HIU-8tdyxYl3ESlSkQu3kCfWBxvZV0#step
Published to http://purl.org/np/RAUod2ToGWqyvBbFqKRPB4ilwjx78fYR1l37ORKDXLsk4
Published concept to http://purl.org/np/RAUod2ToGWqyvBbFqKRPB4ilwjx78fYR1l37ORKDXLsk4#step
Published to http://purl.org/np/RAuKOFZgd1RehE5vOclqdvJo2zchDUUF33VIh8A_8DXtM
Published concept to http://purl.org/np/RAuKOFZgd1RehE5vOclqdvJo2zchDUUF33VIh8A_8DXtM#step
Published to http://purl.org/np/RAFHW530LVNe6DwMV4uvA5cDLItEuvEbHJhfczfW92nMg
Published concept to http://purl.org/np/RAFHW530LVNe6DwMV4uvA5cDLItEuvEbHJhfczfW92nMg#plan


{'nanopub_uri': 'http://purl.org/np/RAFHW530LVNe6DwMV4uvA5cDLItEuvEbHJhfczfW92nMg',
 'concept_uri': 'http://purl.org/np/RAFHW530LVNe6DwMV4uvA5cDLItEuvEbHJhfczfW92nMg#plan'}

Be warned though - the above will keep publishing to the 'real' nanopub server network. For testing you may prefer to publish to the test servers as follows (note that this will refuse to publish a workflow you have already published :

In [10]:
fw.publish_as_nanopub(use_test_server=True)

/Users/robinrichardson/miniconda3/envs/fairworkflows/lib/python3.8/site-packages/fairworkflows/rdf_wrapper.py:213: UserWarning: Cannot publish() this Fair object. This rdf is already published (at http://purl.org/np/RAFHW530LVNe6DwMV4uvA5cDLItEuvEbHJhfczfW92nMg#plan) and has not been modified locally.
  warnings.warn(f'Cannot publish() this Fair object. '


{'nanopub_uri': None, 'concept_uri': None}

You can then find your nanopublications by replacing the base of the URI with http://test-server.nanopubs.lod.labs.vu.nl/

## Execute your workflow using .execute()
Set num_threads greater than 1 if you wish to exploit parallelisation in your workflow. A separate RetroProv object is also returned that can (optionally) be published as a nanopublication.

In [11]:
result, prov = fw.execute(num_threads=2)
result

-66

 ## Retrospective prov
 The retrospective prov object is not yet implemented, pending one of the RDF tasks set out in the planning document. For now, you may see simply the provenance trace for this run. We wish to turn this into a (Plex) RDF nanopublication:

In [12]:
print(prov)

2021-01-24 21:31:39,751 - job            13: add(1, 4)
2021-01-24 21:31:39,751 - job            14: sub(1, 4)
2021-01-24 21:31:39,753 - result         13 [add(1, 4)]: retrieved -> 5
2021-01-24 21:31:39,753 - job            15: weird(5, 3)
2021-01-24 21:31:39,754 - result         14 [sub(1, 4)]: retrieved -> -3
2021-01-24 21:31:39,754 - result         15 [weird(5, 3)]: retrieved -> 22
2021-01-24 21:31:39,755 - job            16: mul(22, -3)
2021-01-24 21:31:39,755 - result         16 [mul(22, -3)]: retrieved -> -66
2021-01-24 21:31:39,755 - -end-of-queue-

